In [4]:
!pip install transformers datasets accelerate openpyxl evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [1]:
from google.colab.files import upload
uploaded=upload()

Saving dataset.xlsx to dataset (1).xlsx


In [7]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import Dataset
import evaluate
from sklearn.model_selection import KFold
import gc

# 1. Load combined dataset
df = pd.read_excel("dataset.xlsx")

# 2. Combine target columns
def make_target(row):
    return f"""Abnormal/Normal: {row['Abnormal/Normal']}
Pathologies Extracted: {row['Pathologies Extracted']}
Midline Shift: {row['Midline Shift']}
Location & Brain Organ: {row['Location & Brain Organ']}
Bleed Subcategory: {row['Bleed Subcategory']}"""

df['target'] = df.apply(make_target, axis=1)

# 3. Load model & tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

max_input_length = 512
max_target_length = 256
device = "cuda" if torch.cuda.is_available() else "cpu"

def preprocess(example):
    input_text = f"Extract info: {example['Radiologist Diagnosis']}"
    input_enc = tokenizer(input_text, padding="max_length", truncation=True, max_length=max_input_length)
    target_enc = tokenizer(example['target'], padding="max_length", truncation=True, max_length=max_target_length)
    input_enc['labels'] = target_enc['input_ids']
    return input_enc

# 4. KFold setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metric = evaluate.load("accuracy")
results = []  # Collect fold results

for fold, (train_index, val_index) in enumerate(kf.split(df), start=1):
    # Split the data
    train_df = df.iloc[train_index].reset_index(drop=True)
    val_df = df.iloc[val_index].reset_index(drop=True)

    # Tokenize and create datasets
    train_dataset = Dataset.from_pandas(train_df[['Radiologist Diagnosis', 'target']]).map(
        preprocess, remove_columns=['Radiologist Diagnosis', 'target'])
    val_dataset = Dataset.from_pandas(val_df[['Radiologist Diagnosis', 'target']]).map(
        preprocess, remove_columns=['Radiologist Diagnosis', 'target'])

    # Reload fresh model per fold
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        return metric.compute(predictions=decoded_preds, references=decoded_labels)

    training_args = TrainingArguments(
        output_dir=f"./t5_fold_{fold}",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=1,  # Lower batch size to save memory
        num_train_epochs=4,
        logging_dir=f"./logs/fold_{fold}",
        logging_steps=10,
        save_total_limit=1,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # Evaluate
    trainer.compute_metrics = None  # Temporarily disable metric computation
    eval_results = trainer.evaluate()



    # Store only required results
    results.append({
        "Fold": fold,
        "Eval Loss": eval_results.get("eval_loss", None),
        "Accuracy": None
    })

    # Aggressive memory cleanup
    del model, trainer, train_dataset, val_dataset
    gc.collect()
    torch.cuda.empty_cache()

# Save final results
results_df = pd.DataFrame(results)
results_df.to_excel("kfold_eval_results.xlsx", index=False)


Map:   0%|          | 0/956 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

/tmp/ipython-input-7-2744255851.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,11.095900
20,6.069100
30,3.381500
40,2.152700
50,1.801700
60,1.492300
70,1.180800
80,0.919400
90,0.720700
100,0.592100


Map:   0%|          | 0/956 [00:00<?, ? examples/s]

...

Step,Training Loss
10,10.543600
20,5.684000
30,2.986200
40,2.071400
50,1.697000
60,1.361900
70,1.135500
80,0.898800
90,0.774900
100,0.633100


Map:   0%|          | 0/956 [00:00<?, ? examples/s]

Map:   0%|          | 0/239 [00:00<?, ? examples/s]

/tmp/ipython-input-7-2744255851.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,10.553400
20,5.432400
30,3.114500
40,2.153000
50,1.722900
60,1.356000
70,1.073200
80,0.865600
90,0.723300
100,0.582200


In [12]:
model = T5ForConditionalGeneration.from_pretrained("./t5_fold_4/checkpoint-480")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Save to a new directory
save_path = "./t5_radiology_final"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./t5_radiology_final/tokenizer_config.json',
 './t5_radiology_final/special_tokens_map.json',
 './t5_radiology_final/spiece.model',
 './t5_radiology_final/added_tokens.json')